# Assignment 3: Improving the Index

In this assignment, we will improve the search index and query functions from the previous assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10, sqrt

Summaries_file = 'data/cognition_Summaries.pkl.bz2'
Abstracts_file = 'data/cognition_Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

print(Summaries[33621483])

paper(title='Social behaviors and gray matter volumes of brain areas supporting social cognition in children and adolescents with prenatal alcohol exposure.', authors=['de Water E', 'Rockhold MN', 'Roediger DJ', 'Krueger AM', 'Mueller BA', 'Boys CJ', 'Schumacher MJ', 'Mattson SN', 'Jones KL', 'Lim KO', 'Wozniak JR'], year=2021, doi='10.1016/j.brainres.2021.147388')


In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [4]:
inverted_index = defaultdict(list)

for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. We won't go much into the details of tokenization and linguistic analysis here, because we also want to focus on scoring and ranking below. Therefore, we are using an existing library for tokenizatoin and stemming, namely the NLTK package. The following line will install NLTK if necessary (or you have to follow [these instructions](http://www.nltk.org/install.html) if that doesn't work):

In [5]:
! pip install --user nltk

In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
stemmer = EnglishStemmer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print('INPUT TEXT:\n ', s)

print('TOKENIZE: ', tokenize(s))
print('WORD TOKENIZE: ', word_tokenize(s))

[nltk_data] Downloading package punkt to /Users/riles/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


INPUT TEXT:
  Good muffins cost $3.88
in New York.  Please buy me two of them.

Thanks.
TOKENIZE:  ['Good', 'muffins', 'cost', '$3.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
WORD TOKENIZE:  ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [15]:
print(stemmer.stem("processes"))
print(stemmer.stem("apples"))

process
appl


## Ranking

Another important method to improve our search results is to rank them, which can be done by calculating a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, which comes with several variants, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- num_documents: The number of documents in our index

In [8]:
tf_matrix = defaultdict(Counter)

for doc_id in Summaries.keys():
    tokens = preprocess(tokenize(Summaries[doc_id].title))
    if (doc_id in Abstracts):
        tokens.extend(preprocess(tokenize(Abstracts[doc_id])))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Summaries))

Let's test these functions with some examples:

In [9]:
print(tf('amsterdam', 20594617))
print(df('amsterdam'))
print(num_documents)

1.0
35.0
102110.0


With these helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

# Tasks

**Your name:** ...

### Task 1

Implement in the code block below the `smarter_tokenize_and_preprocess` function using NLTK's functions for tokenization and stemming. 

In [14]:
# Your code here:
def smarter_tokenize_and_preprocess(text):
    split_tokens = word_tokenize(text);
    
    result = []
    for token in split_tokens:
        result.append(stemmer.stem(token.lower()))
    return result
    

print(smarter_tokenize_and_preprocess("where are My apples!"))

['where', 'are', 'my', 'appl', '!']


Now we can make a smarter index based on this function. For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time. (You don't need to change or add anything in the code block below. Just leave it as it is.)

In [17]:
# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(list)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = list(key for key in Summaries.keys() if 33000000 <= key < 34000000)

# Building our smarter index:
for id in sorted(subset_of_ids):
    term_set = set(smarter_tokenize_and_preprocess(Summaries[id].title))
    if id in Abstracts:
        term_set.update(smarter_tokenize_and_preprocess(Abstracts[id]))
    for term in term_set:
        smarter_index[term].append(id)

Now implement the `smarter_and_query` function, based on the function `smarter_tokenize_and_preprocess` you defined above and accessing our new index `smarter_index`. You can start from copying the code for `and_query` from the last assignment. For that to work, you'll also have to copy the code for the `and_merge` function from the last assignment.

In [28]:
# Smarter and_query based on the smarter tokenize and preprocess functions

def and_merge(sorted_list1, sorted_list2):  
    merged_list = []  
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)  
    list2 = list(sorted_list2)  
    while (list1 and list2):  
        if (list1[0] < list2[0]):  
            list1.pop(0)  
        elif (list1[0] > list2[0]):  
            list2.pop(0)  
        else:  
            merged_list.append(list1[0])  
            list1.pop(0)  
            list2.pop(0)  
    return merged_list  

def smarter_and_query(query_string):  
    query_words = smarter_tokenize_and_preprocess(query_string)
    first_word = query_words[0]  
    remaining_words = query_words[1:]  
    and_list = smarter_index[first_word]  
    for t in remaining_words:  
        and_list = and_merge(and_list, smarter_index[t])  
    return and_list  

### Task 2

Run the query "automated quality controls" with the new `smarter_and_query` function from task 1. Does it return paper *33621483*? Explain what our new smarter function specifically contributes to the result (as compared to our previous naive implementations for tokenization and preprocessing)?

In [36]:
#smarter_and_query("quality controls checklist")
print(smarter_tokenize_and_preprocess("automated quality controlling"))
print(smarter_and_query("automated quality controls"))
display_summary(33621483, show_abstract=True, show_id=True)

['autom', 'qualiti', 'control']
[33368865, 33482395, 33621483]


**Answer:** 

Paper with ID 33621483 contains the words: "automated", "quality", and "controlling", which get stemmed to:'autom', 'qualiti', and 'control', respectively. This is the same as what the query "automated quality controls" gets tokenized to. Therefore it will show up in the smarter_and_query

### Task 3

Now we move to a different subject and use our old index again. That is, we **don't** use the smarter functions defined above for tasks 3 to 5!

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using `tf(t,d)`, `df(t)` and `num_documents` as defined above. To do this, first implement a function `idf(t)` to calculate the inverse document frequency, and then use this function to calculate the full tf-idf. Use the _add-one-smoothing_ version of idf, so we don't run into problems with terms that don't appear in the collection at all. The relevant formulas can be found on the lecture slides. Use tf-idf with plain (non-logarithmic) term frequency, as applied by scoring variant `ntn`. Test your function with the examples shown below. You can use the `log10(n)` function to calculate the base 10 logarithm.

Again, use our old (non-smart) index for this task and the tasks below, and **not** the functions defined in tasks 1 and 2.

In [46]:
# Your code here:
def tfidf(t,d):
    return tf(t, d)*idf(t);

def idf(t):
    return log10((num_documents+1)/(df(t)+1));

print(tfidf("amsterdam", 20594617))

#come back here to check that the smoothing is right
#print(df(20594617));

3.4527700286063565


### Task 4

Create a function `query_ntn_nnn(query_string)`, which accepts as input a single query string of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying variant `ntn.nnn`, as above (see the formula for the `ntn.nnn` version of scoring on the lecture slides). Use an auxiliary function `score_ntn_nnn` to calculate the score. The results should be shown in descending order by score.

You can start by copying your functions `or_merge` and `or_query` from assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created above.

Demonstrate your function by giving it the exemplary query string "happiness and sadness of young adults".

In [66]:
# Your code here:
def query_ntn_nnn(query_string):
    #create default dict with key = word in query, value = ntn.nnn score
    document_scores = defaultdict(int);
    
    #for each document:
    for doc_id in or_query(query_string):
        #calculate total score
        document_scores[doc_id] = score_ntn_nnn(query_string, doc_id);

    return sorted(document_scores.items(), key=lambda i:i[1], reverse=True)[:10]

def score_ntn_nnn(q, d):
    cur_score = 0;
    for term in preprocess(tokenize(q)):
        cur_score+=tfidf(term, d);
    return cur_score;






def or_query(inputString):
    tokenizedInput = preprocess(tokenize(inputString));
    
    matchingDocuments = inverted_index[tokenizedInput[0]];
    
    for x in range(len(tokenizedInput)-1):
        matchingDocuments = or_merge(matchingDocuments, inverted_index[tokenizedInput[x+1]]);
    
    return matchingDocuments;

def or_merge(l1, l2):
    combined = [];
    index1 = 0;
    index2 = 0;

    while(index1 < len(l1) and index2 < len(l2)):
        if(l1[index1] < l2[index2]):
            combined.append(l1[index1]);
            index1+=1;
        elif(l2[index2] < l1[index1]):
            combined.append(l2[index2])
            index2+=1;
        else:
            combined.append(l1[index1])
            #increment both
            index1+=1;
            index2+=1;

        #add all remaining values
    while(index1 < len(l1)):
        combined.append(l1[index1]);
        index1+=1;
    while(index2 < len(l2)):
        combined.append(l2[index2]);
        index2+=1;

    return combined;    



In [67]:
print(query_ntn_nnn("happiness and sadness of young adults"))
#print(query_ntn_nnn("amsterdam"))


[(24046755, 21.74695831423794), (24628461, 20.713595738895215), (31817633, 20.425747317841044), (35095690, 19.866026923007695), (19389462, 19.85220333774405), (34899267, 17.92865427406855), (32038362, 16.6102165732608), (24970235, 16.348395873280644), (35782096, 15.586692232912489), (31865171, 15.47475995098112)]


In [68]:
display_summary(24046755, show_abstract=True, show_id=True)

### Task 5

In this last task, you should create a second version of the query function from Task 4, called `query_ntc_ntc`. This second version should use, as its name suggests, variant `ntc.ntc` instead of `ntn.nnn`, and therefore apply the cosine similarity measure, in addition to applying _tf-idf_. For this, consult the formula for variant `nnc.nnc` on the lecture slides and adopt it to include the _idf_ metric (that is, add the `t` element of `ntc`). (You can drop the square root of |q| in the formula, as indicated on the slides.)

As a first step, we can calculate beforehand the length of all document vectors (because they don't depend on the query) for document vectors consisting of _tf-idf_ values. The code below does just that, assuming that you defined the function `tfidf(t,d)` above (don't change this code block, just run it):

In [73]:
tfidf_length_values = defaultdict(int)

for doc_id in Summaries.keys():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

We can now get the length of a document vector by calling `tfidf_length(d)`.

Based on this, you can now implement `query_ntc_ntc` in the code block below. You should again first define an auxiliary function, called `score_ntc_ntc`. You can start by copy-pasting the code from Task 4.

To output the results, use the provided `display_summary` function to make the output a bit more like the results page of a search engine. Lastly, demonstrate your `query_ntc_ntc` function with the same example query as above: "happiness and sadness of young adults"

In [88]:
# Your code here:
def query_ntc_ntc(query_string):
    #create default dict with key = word in query, value = ntn.nnn score
    document_scores = defaultdict(int);
    
    #for each document:
    for doc_id in or_query(query_string):
        #calculate total score
        document_scores[doc_id] = score_ntc_ntc(query_string, doc_id);

    top_10_list = sorted(document_scores.items(), key=lambda i:i[1], reverse=True)[:10]

    for pair in top_10_list:
        display_summary(pair[0], show_abstract=True, show_id=True)

def score_ntc_ntc(q,d):
    cur_score = 0;
    for term in preprocess(tokenize(q)):
        cur_score+=(tfidf(term, d)/tfidf_length(d));
    return cur_score;
    
query_ntc_ntc("happiness and sadness of young adults")

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.